In [1]:
import traci
import sumolib

In [3]:
sumoBinary = "sumo"
sumoCmd = [sumoBinary, "-c", "nt1.sumocfg"]

traci.start(sumoCmd)

TraCIException: Connection 'default' is already active.

In [ ]:
tls_ids = traci.trafficlight.getIDList()
print("Semáforos", tls_ids)

In [ ]:
id_semaforo = tls_ids[0]
programa = traci.trafficlight.getAllProgramLogics(id_semaforo)
print(programa)

In [ ]:
from traci._trafficlight import Logic, Phase

#Crear un nuevo programa con fases personalizadas




new_logic = Logic(programID="nuevo_programa", type=0,currentPhaseIndex=0,phases=[Phase(duration=30, state="GGrr", minDur=30, maxDur=30 ), Phase(duration=5, state='yyrr', minDur=5, maxDur=5), Phase(duration=30, state='rrGG', minDur=30, maxDur=30), Phase(duration=5, state='rryy', minDur=5, maxDur=5), Phase(duration=1, state='rrrr', minDur=1.0, maxDur=1.0)])

#Aplicar el nuevo programa al semáforo
traci.trafficlight.setProgramLogic(id_semaforo, new_logic)


traci.trafficlight.setProgram(id_semaforo, "nuevo_programa")

In [10]:
def new_logic(ggrr, yyrr, rrgg, rryy):

    tls_ids = traci.trafficlight.getIDList()
    id_semaforo = tls_ids[0]
    programa = traci.trafficlight.getAllProgramLogics(id_semaforo)
    
    from traci._trafficlight import Logic, Phase

    new_logic = Logic(programID="nuevo_programa", type=0,currentPhaseIndex=0,phases=[Phase(duration=ggrr, state="GGrr", minDur=ggrr, maxDur=ggrr ), Phase(duration=yyrr, state='yyrr', minDur=yyrr, maxDur=yyrr), Phase(duration=rrgg, state='rrGG', minDur=rrgg, maxDur=rrgg), Phase(duration=rryy, state='rryy', minDur=rryy, maxDur=rryy)])

    traci.trafficlight.setProgramLogic(id_semaforo, new_logic)

    traci.trafficlight.setProgram(id_semaforo, "nuevo_programa")

    print(traci.trafficlight.getCompleteRedYellowGreenDefinition(id_semaforo))


In [ ]:
new_logic(30,5,30,5,1)

In [11]:
def avg_time():
    depart_times = {}
    arrival_times = {}
    simulation_durations = {}

    for step in range(3600):
        traci.simulationStep()

        departed_vehicles = traci.simulation.getDepartedIDList()
        current_time = traci.simulation.getTime()
        for veh_id in departed_vehicles:
            if veh_id not in depart_times:
                depart_times[veh_id] = current_time
        
        

        arrived_vehicles = traci.simulation.getArrivedIDList()
        current_time = traci.simulation.getTime()
        for veh_id in arrived_vehicles:
            if veh_id not in arrival_times:
                arrival_times[veh_id] = current_time



    for veh_id, arrival_time in arrival_times.items():
        if veh_id in depart_times:
            departure_time = depart_times[veh_id]
            duration = arrival_time - departure_time
            simulation_durations[veh_id] = duration

    average = sum(simulation_durations.values())/len(simulation_durations)

    return average

In [12]:
#Se importan algunos de los paquetes requeridos
import random as rd
import pandas as pd
import traci
import sumolib

avgtimes=[]#tiempos promedio
ggrrs = []#(verde Norte-Sur)
yyrrs = []#(amarillo Norte-Sur)
rrggs = []# (verde Este-Oeste)
rryys = []#(amarillo Este-Oeste)
num_simulations = 25

for i in range(num_simulations):
    sumoBinary = "sumo"
    sumoCmd = [sumoBinary, "-c", "nt1.sumocfg"]

    traci.start(sumoCmd)

    ggrr = rd.uniform(10,60) #(verde Norte-Sur) 
    yyrr = rd.uniform(2,5) #(amarillo Norte-Sur)
    rrgg = rd.uniform(10,60) # (verde Este-Oeste)
    rryy = rd.uniform(2,5) #(amarillo Este-Oeste)

    ggrrs.append(ggrr)
    yyrrs.append(yyrr)
    rrggs.append(rrgg)
    rryys.append(rryy)

    new_logic(ggrr, yyrr, rrgg, rryy)
    depart_times = {}
    arrival_times = {}
    simulation_durations = {}

    for step in range(3600):
        traci.simulationStep()

        departed_vehicles = traci.simulation.getDepartedIDList()
        current_time = traci.simulation.getTime()
        for veh_id in departed_vehicles:
            if veh_id not in depart_times:
                depart_times[veh_id] = current_time
        
        

        arrived_vehicles = traci.simulation.getArrivedIDList()
        current_time = traci.simulation.getTime()
        for veh_id in arrived_vehicles:
            if veh_id not in arrival_times:
                arrival_times[veh_id] = current_time



    for veh_id, arrival_time in arrival_times.items():
        if veh_id in depart_times:
            departure_time = depart_times[veh_id]
            duration = arrival_time - departure_time
            simulation_durations[veh_id] = duration

    average = sum(simulation_durations.values())/len(simulation_durations)

    avgtimes.append(average)

    traci.close()
data = {
    'Simulation' : range(1, num_simulations + 1),
    'Tiempo promedio' : avgtimes,
    'GGrr (s)' : ggrrs,
    'yyrr (s)' : yyrrs,
    'rrGG (s)' : rrggs,
    'rryy (s)' : rryys
}

df = pd.DataFrame(data)
df

 Retrying in 1 seconds


C:\Users\Erasmo\AppData\Local\Temp\ipykernel_12260\193906455.py:15: UserWarning: Call to deprecated function getCompleteRedYellowGreenDefinition, use getAllProgramLogics instead.
  print(traci.trafficlight.getCompleteRedYellowGreenDefinition(id_semaforo))


(Logic(programID='0', type=0, currentPhaseIndex=0, phases=(Phase(duration=15.0, state='GGrr', minDur=15.0, maxDur=15.0), Phase(duration=3.0, state='yyrr', minDur=3.0, maxDur=3.0), Phase(duration=15.0, state='rrGG', minDur=15.0, maxDur=15.0), Phase(duration=3.0, state='rryy', minDur=3.0, maxDur=3.0)), subParameter={}), Logic(programID='nuevo_programa', type=0, currentPhaseIndex=0, phases=(Phase(duration=26.25, state='GGrr', minDur=26.25, maxDur=26.25), Phase(duration=3.408, state='yyrr', minDur=3.408, maxDur=3.408), Phase(duration=15.043, state='rrGG', minDur=15.043, maxDur=15.043), Phase(duration=2.944, state='rryy', minDur=2.944, maxDur=2.944)), subParameter={}))
(Logic(programID='0', type=0, currentPhaseIndex=0, phases=(Phase(duration=15.0, state='GGrr', minDur=15.0, maxDur=15.0), Phase(duration=3.0, state='yyrr', minDur=3.0, maxDur=3.0), Phase(duration=15.0, state='rrGG', minDur=15.0, maxDur=15.0), Phase(duration=3.0, state='rryy', minDur=3.0, maxDur=3.0)), subParameter={}), Logic(p

,Simulation,Tiempo promedio,GGrr (s),yyrr (s),rrGG (s),rryy (s)
0,1,43.541667,26.250395,3.408231,15.043018,2.943994
1,2,39.770933,18.766916,2.926916,15.861111,3.897501
2,3,39.414690,31.581841,4.158310,46.225701,3.769942
3,4,39.043241,40.534278,2.500302,42.777850,2.663379
4,5,42.265257,46.084134,2.895202,43.821433,4.834333
5,6,44.593120,58.456338,3.724866,25.787344,4.818257
6,7,55.024661,10.705911,4.536342,59.927753,3.500509
7,8,42.694336,54.383737,2.629352,45.123595,2.945000
8,9,35.734056,15.572436,3.006405,26.986008,2.693758
9,10,46.131937,58.774662,3.443436,57.528233,2.949990


In [13]:
df.to_csv('Nuevos_valores2.csv', index=False)

In [14]:
df2 = pd.read_csv('Nuevos_valores.csv')
df3 = pd.read_csv('Nuevos_valores2.csv')

df_test = pd.concat([df2, df3], ignore_index=True)

In [15]:
df_test

,Simulation,Tiempo promedio,GGrr (s),yyrr (s),rrGG (s),rryy (s)
0,1,42.273792,54.379963,2.275228,45.521003,2.452633
1,2,47.816987,21.506975,4.334561,10.419554,2.988516
2,3,37.672950,32.006911,3.209169,39.412099,4.236100
3,4,41.208770,41.769052,2.230724,51.537889,3.413446
4,5,44.885861,53.764795,2.604055,58.150533,2.981233
...,...,...,...,...,...,...
120,21,36.594094,29.795930,2.934714,42.411125,2.958799
121,22,38.672269,15.532804,2.468803,35.100084,3.105625
122,23,35.886336,27.340851,3.555902,24.147868,3.140026
123,24,38.795082,30.466068,4.343339,47.943494,2.271605


In [17]:
df_test.to_csv('df_test.csv', index=False)